In [ ]:
!pip install vantage6-client

In [ ]:
# Setup, run once
from vantage6.client import Client

In [ ]:
client = Client("http://localhost", 5000, "/api")
client.authenticate("node1-admin", "password")
client.setup_encryption(None)

In [ ]:
# Find out the organizations and collaborations we can run on
import json
collaboration_list = client.collaboration.list()
collaboration_index = 0
organization_ids_ = [ ]

for organization in collaboration_list[collaboration_index]['organizations']:
    organization_ids_.append(organization['id'])

In [ ]:
# Write your query here
query = """
prefix snomedct: <http://purl.bioontology.org/ontology/SNOMEDCT/> 
prefix roo: <http://www.cancerdata.org/roo/>
prefix ncit: <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#>

select ?patient ?data where { 
    ?patient a snomedct:108369006 .
	?patient roo:P100214 ?measurement .
    ?measurement a ncit:C106303 .
    ?measurement roo:P100042 ?data .
}
"""

query = """
prefix snomedct: <http://purl.bioontology.org/ontology/SNOMEDCT/>
prefix roo: <http://www.cancerdata.org/roo/>
prefix ncit: <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>




select ?patientIdentifier ?age ?sex ?diseaseDate ?location ?tumourType



where {
?patient a snomedct:116154003;
ncit:P395 ?patientIdentifier. #targetCode
?patient roo:P100000 ?ageClass. #has age
?ageClass roo:P100042 ?age.
?patient roo:P100018 ?sexClass. #has biological sex
?sexClass roo:P100042 ?sexValue.
BIND( xsd:string(IF(?sexValue = 2, "female", "male")) AS ?sex).
?patient roo:P100008 ?tumour.#has neoplasm
?tumour ncit:P395 ?diseaseEpisode;
roo:P100042 ?tumourTypeClass;
roo:P100257 [ #has disease date
roo:P100042 ?diseaseDate];
roo:P100202 [ #has location
rdf:type snomedct:272451001;
roo:P100042 ?locationClass
].
BIND (COALESCE(
IF(?locationClass = "C500", "Areola", 1/0),
IF(?locationClass = "C501", "Central Part", 1/0),
IF(?locationClass = "C502", "medial upper quadrant", 1/0),
IF(?locationClass = "C503", "medial lower quadrant", 1/0),
IF(?locationClass = "C504", "lateral upper quadrant", 1/0),
IF(?locationClass = "C505", "lateral lower quadrant", 1/0),
IF(?locationClass = "C506", "axillary tail", 1/0),
IF(?locationClass = "C508", "overlap", 1/0),
IF(?locationClass = "C509", "unspecified", 1/0),
"F") AS ?location)
BIND (COALESCE(
IF(?tumourTypeClass = 501300, xsd:string("Invasive mammary carcinoma"), 1/0),
IF(?tumourTypeClass = 502200, xsd:string("Ductal carcinoma in situ"), 1/0),
IF(?tumourTypeClass = 503200, xsd:string("Lobular carcinoma in situ"), 1/0),
"F" ) AS ?tumourType)

}"""

In [ ]:
# Request the structure to be built
input_ = {
    "master": "true",
    "method":"master", 
    "args": [ ],
    "kwargs": {
        "query": query
    }
}

task = client.post_task(
    name="ExploreData",
    image="jaspersnel/v6-summary-py-sparql:swat4ls-0.1.0",
    collaboration_id=collaboration_list[collaboration_index]['id'],#Get the first collaboration associated with user
    input_= input_,
    organization_ids=[organization_ids_[0]]
)

print(json.dumps(task, indent=2))

In [ ]:
# Wait for results
import time
import json
resultObjRef = task.get("results")[0]
resultObj = client.result.get(resultObjRef['id'])
attempts = 1
while((resultObj["finished_at"] == None) and attempts < 10):
    print("waiting...")
    time.sleep(5)
    resultObj = client.result.get(resultObjRef['id'])
    attempts += 1
results = resultObj['result']

print(results)